In [1]:
import numpy as np
import matplotlib.pylab as plt
import corner
import numpy as np
import glob
from PTMCMCSampler import proposals

%matplotlib inline

## List of available jump proposals

`PTMCMC` already has a list of default jump proposals. These can be viewed by running

In [2]:
proposals.available_jump_proposals()

Available jump proposals:

SingleComponentAdaptiveCovariance
AdaptiveCovariance
SingleComponentAdaptiveGaussian
MultiComponentAdaptiveGaussian
AdaptiveGaussian
DifferentialEvolution
Normal
Uniform
Prior


## Default jump proposals

By default the following jump proposals will be used

In [3]:
proposals.default_jump_proposals()

Default jump proposals:

SingleComponentAdaptiveCovariance
AdaptiveCovariance
SingleComponentAdaptiveGaussian
MultiComponentAdaptiveGaussian
AdaptiveGaussian
DifferentialEvolution


Each jump proposal will have an equal weighting. This means that each jump proposal has equal probability of being picked.

## Choosing specific jump proposals

Of course, having equal weighting for all jump proposals might not suit your problem. You are able to specify different weights for different jump proposals by passing a dictionary called weights to the `sampler` object. This dictionary might have the following form:

In [4]:
weights = {"SingleComponentAdaptiveCovariance": 5,
           "AdaptiveCovariance": 1,
           "SingleComponentAdaptiveGaussian": 1,
           "MultiComponentAdaptiveGaussian": 1,
           "AdaptiveGaussian": 1,
           "DifferentialEvolution": 1}

Here, the `SingleComponentAdaptiveCovariance` jump proposal will have a probaility of being used equal to `5/10 = 1/2` while the other jump proposals have probability of being chosen equal to `1/10`.

If you only want to use the `SingleComponentAdaptiveCovariance` jump proposal, then this can be done by writing your weight dictionary as:

In [5]:
weights = {"SingleComponentAdaptiveCovariance": 5}

Some jump proposals (like `Normal` and `Uniform`) require addtional keyword arguments. These keyword arguments are passed to the `sampler` object by using the `jump_proposal_arguments` keyword argument. For example, if you want to use the `Uniform` jump proposal with bounds of `0` and `100` you would define a jump_proposal_argument dictionary as:

In [6]:
jump_proposal_arguments = {"Uniform": {"pmin": 0, "pmax": 100}}

## Custom jump proposals

Custom jump proposals can be added very easily